# Recap Data Challenge

1. Fetch all invoices and save to file
2. EDA and Transformation to handle **net revenue** and **churned amount**
3. Refactor into .py scripts
4. Add tests + possibly CI

## Fetch all invoices

In [1]:
import requests
import json

In [2]:
BASE_URL = "https://nookdtmzylu7w75p7atatnzom40zmdpz.lambda-url.eu-central-1.on.aws/invoices"

In [17]:
def fetch_all_invoices():
    invoices = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        response = requests.get(f"{BASE_URL}?page={page}")
        response_data = response.json()

        #break if we get anything other than 200 repsonse
        if response_data.get('status_code') != 200:
            break
        
        data = response_data.get('body', {}).get('data', [])
        total_pages = response_data.get('body', {}).get('total_pages', 1)

        #break once we've reach max pages
        if not data and page > total_pages:
            break
            
        print(f"Finished fetching page {page}")
        invoices.extend(data)
        page += 1
    
    return invoices

In [18]:
invoices = fetch_all_invoices()

Finished fetching page 1
Finished fetching page 2
Finished fetching page 3
Finished fetching page 4
Finished fetching page 5
Finished fetching page 6
Finished fetching page 7
Finished fetching page 8
Finished fetching page 9
Finished fetching page 10


In [96]:
# Save invoices to a file
with open("data/invoices.json", "w") as f:
    json.dump(invoices, f, indent=4)

## Compute the net revenue and churned amount for each contract in each month

In [20]:
import pandas as pd

In [73]:
# Load the invoices from the file
with open("invoices.json") as f:
    invoices = json.load(f)

In [74]:
# Convert invoices to a DataFrame
df = pd.DataFrame(invoices)
df.head(20)

,original_billing_amount,contract_id,invoice_id,invoice_date
0,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,1abe8fb1b7c47cff00a3c8b6a20e989f273b2a73514ffd...,2021-09-01
1,-500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,f801b069a7777f6bb357f7d5afa2d70c74a8945f251ca3...,2021-08-01
2,-500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,bdea40f6f3f2ca47a2eb185cd13eaced10f7a5ba69cde6...,2021-07-01
3,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,aae0890f787eb93c07a7e01c260aa1094d6a2a28f72f05...,2021-06-01
4,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,0fcc07c8215509ed852e3ad32e2c5f74806d7216018e0f...,2021-05-01
5,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,9be7b5fea63ee8a07c126ba311539fea4d07ed6563e116...,2021-04-01
6,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,c963887fcf9028f6c8878ea1359eeaebc7613cc874c8ef...,2021-01-01
7,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,cee345a51d8befdfa0c15ec35a8092f0ffacb3962b35a4...,2020-12-01
8,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,349b130ab5a7c873bac0a32b099041a3c1c81230c5497e...,2020-11-01
9,500.00,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,4d8c45ce7843789572ee264def078cf77831896997fbfd...,2020-10-01


In [75]:
# Convert invoice_date to datetime
df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [76]:
df['month'] = df['invoice_date'].dt.to_period('M')

In [77]:
df = df.groupby(['contract_id', 'month']).agg({'original_billing_amount': 'sum'}).reset_index()
df.rename(columns={'original_billing_amount': 'net_revenue'}, inplace=True)

In [78]:
df = df.sort_values(by=['contract_id', 'month'])

In [79]:
df

,contract_id,month,net_revenue
0,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-09,216.67
1,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-10,500.00
2,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-11,500.00
3,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-12,500.00
4,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2021-01,500.00
...,...,...,...
457,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-05,1425.62
458,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-06,1425.62
459,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-07,1425.62
460,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-08,1425.62


In [80]:
df['previous_net_revenue'] = df.groupby('contract_id')['net_revenue'].shift().fillna(0)

In [81]:
df

,contract_id,month,net_revenue,previous_net_revenue
0,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-09,216.67,0.00
1,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-10,500.00,216.67
2,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-11,500.00,500.00
3,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-12,500.00,500.00
4,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2021-01,500.00,500.00
...,...,...,...,...
457,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-05,1425.62,1425.62
458,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-06,1425.62,1425.62
459,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-07,1425.62,1425.62
460,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-08,1425.62,1425.62


In [82]:
# Calculate churned amount
df['churned_amount'] = df.apply(
        lambda row: row['previous_net_revenue'] - row['net_revenue'] if row['previous_net_revenue'] > row['net_revenue'] else 0,
        axis=1
    )

In [83]:
df

,contract_id,month,net_revenue,previous_net_revenue,churned_amount
0,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-09,216.67,0.00,0.0
1,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-10,500.00,216.67,0.0
2,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-11,500.00,500.00,0.0
3,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-12,500.00,500.00,0.0
4,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2021-01,500.00,500.00,0.0
...,...,...,...,...,...
457,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-05,1425.62,1425.62,0.0
458,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-06,1425.62,1425.62,0.0
459,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-07,1425.62,1425.62,0.0
460,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-08,1425.62,1425.62,0.0


In [84]:
df = df.drop(columns=['previous_net_revenue'])

In [86]:
df['net_revenue'] = df['net_revenue'].round(2)
df['churned_amount'] = df['churned_amount'].round(2)

In [87]:
df

,contract_id,month,net_revenue,churned_amount
0,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-09,216.67,0.0
1,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-10,500.00,0.0
2,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-11,500.00,0.0
3,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2020-12,500.00,0.0
4,00526b90f38d73b0138904f8df7353c456e28ae87fbe89...,2021-01,500.00,0.0
...,...,...,...,...
457,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-05,1425.62,0.0
458,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-06,1425.62,0.0
459,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-07,1425.62,0.0
460,0c2ed1034b171c5f5a0212321923234eacf93fb22941c6...,2021-08,1425.62,0.0


# Save final DF (Task 2) to a csv file

In [100]:
def save_net_revenue_churn_to_file(df, filename="data/net_revenue_churn.csv"):
    df.to_csv(filename, index=False)

In [101]:
save_net_revenue_churn_to_file(df)

## EDA

In [88]:
df['churned_amount'].value_counts()

churned_amount
0.00       409
66.45        1
141.23       1
1019.09      1
107.52       1
59.50        1
226.41       1
1698.00      1
65.50        1
174.00       1
232.31       1
53.48        1
203.33       1
547.93       1
1000.00      1
337.35       1
117.81       1
461.68       1
76.24        1
692.52       1
11.74        1
217.06       1
1887.27      1
7198.73      1
996.00       1
178.50       1
69.09        1
1535.36      1
522.41       1
0.01         1
347.42       1
326.94       1
355.81       1
1785.00      1
235.74       1
337.73       1
578.84       1
8032.50      1
119.00       1
236.81       1
63.15        1
173.66       1
580.00       1
587.50       1
595.00       1
238.00       1
693.66       1
1499.40      1
135.90       1
399.74       1
376.58       1
124.66       1
623.75       1
1094.71      1
Name: count, dtype: int64

In [102]:
churn_sum = df.groupby('contract_id')['churned_amount'].sum().reset_index()

# Sort by churned_amount in descending order
churn_sum_sorted = churn_sum.sort_values(by='churned_amount', ascending=False)

In [103]:
churn_sum_sorted

,contract_id,churned_amount
14,027b2bd7b7e279f08dd537e8079b20fd3ed8902f2ff196...,8032.50
62,0b3951ad0b09487177c1c624d8d94f7a8a26169a1fd79c...,7198.73
61,0af503a6ad939a715f74c281b6812381f673b1f339e2a1...,2104.33
21,045a46c96dc19f0dc3135fc0266ac7cadfca1540f26f66...,2000.50
49,08ca5796af365ed726244d7061c0abb9437427ff3aa658...,1924.41
...,...,...
39,063afa4fa05640f71bc203655ff84bc81146cd03da566b...,0.00
13,021869529aa5dd14e778dba791308d5def40005b1595c9...,0.00
41,0740a894ce491999d0807ce315468035baf42f76d7f5da...,0.00
42,07963ddebac9417cd1fa40af4a006f45a56216945d8e51...,0.00
